In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [4]:
independent=dataset[["Age",	"EstimatedSalary",	"Gender_Male"]]
dependent=dataset["Purchased"]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [9]:
#Preprocessing
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],
    'penalty': ['l1', 'l2', 'elasticnet']  # Correct values for the penalty parameter
}

grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fitting the model
grid.fit(X_train, Y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [14]:
#print best parameter
#TestSet can be predicted by final model
result=grid.cv_results_
grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)

from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

In [15]:
from sklearn.metrics import f1_score
f1_macro = f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l1', 'solver': 'saga'}: 0.8794289739855382


In [16]:
print("Confusion Matrix:\n",cm)

Confusion Matrix:
 [[79  6]
 [10 39]]


In [17]:
print("The classification report:\n",clf_report)

The classification report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [21]:
#roc=r for receiver, o for operating, c for curve: auc= a for area, u for under, c for curve
from sklearn.metrics import roc_auc_score
#If probability is not given, then it won't work
roc_auc_score(Y_test, grid.predict_proba(X_test)[:,1])

0.9493397358943578

In [23]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002006,0.001385,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,0.002035,0.000861,0.000000,0.000000,l1,lbfgs,"{'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.013791,0.003611,0.014114,0.000403,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.804764,0.811321,0.66593,0.92351,0.922185,0.825542,0.094918,6
3,0.008086,0.003491,0.006807,0.001567,l1,saga,"{'penalty': 'l1', 'solver': 'saga'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
4,0.014857,0.004994,0.005424,0.000760,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
5,0.012784,0.007679,0.006540,0.000909,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
6,0.004667,0.001908,0.004730,0.000179,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
7,0.004027,0.001071,0.005922,0.000951,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,3
8,0.001098,0.000195,0.000000,0.000000,elasticnet,newton-cg,"{'penalty': 'elasticnet', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.001202,0.000406,0.000000,0.000000,elasticnet,lbfgs,"{'penalty': 'elasticnet', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
